In [2]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2

In [3]:
from src.constants import GEDI_L2A_PATH
from src.utils.os import list_content
from src.utils.download import download
from tqdm.autonotebook import tqdm
import geopandas as gpd
save_dir = GEDI_L2A_PATH/ "v002" / "amazon_basin"

<ipython-input-3-f1060345d3a8>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
feather_files = list(save_dir.glob("*/*.feather"))
print(f"There are {len(feather_files)} feather files.")

There are 5193 feather files.


## Count number of total shots

Takeaway:  
>It takes about 2-3 seconds to read a simple feather dataframe into geopandas.  
This means in total it will take about 3-4h to read in all the data.

> From a rough look at the first 100 samples, there will be about 500 Mio - 1 B shots over the Amazon.  
> Exact number: 452'202'228 (450 Mio.)

Note: if we just want to get the lenght, we can also read via pandas:

In [ ]:
import pandas as pd

n_shots = 0
for feather in tqdm(feather_files):
    n_shots += len(pd.read_feather(feather, columns=["quality_flag"]))
    
print(n_shots)

In [5]:
print(n_shots)

452202228


## Look at a sample of the dataset

In [5]:
feather_files[0].stat().st_size / 1024 / 1024 

20.070505142211914

In [7]:
sample = gpd.read_feather(feather_files[0])
sample.head()

,granule_name,shot_number,beam_type,beam_name,delta_time,absolute_time,sensitivity,quality_flag,solar_elevation,solar_azimuth,...,rh92,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100,geometry
0,GEDI02_A_2019108093620_O01965_01_T05338_02_003...,19650000100036037,coverage,BEAM0000,4.081639e+07,2019-04-18 09:53:13.286247648+00:00,0.683428,1,-15.522217,-15.522217,...,1.72,1.76,1.83,1.94,2.02,2.09,2.21,2.36,2.51,POINT (-71.12366 -18.09552)
1,GEDI02_A_2019108093620_O01965_01_T05338_02_003...,19650000100036038,coverage,BEAM0000,4.081639e+07,2019-04-18 09:53:13.294511653+00:00,-0.907164,0,-15.521826,-15.521826,...,1.46,1.53,1.57,1.64,1.72,1.79,1.87,1.98,2.09,POINT (-71.12333 -18.09511)
2,GEDI02_A_2019108093620_O01965_01_T05338_02_003...,19650000100036039,coverage,BEAM0000,4.081639e+07,2019-04-18 09:53:13.302775651+00:00,0.614895,1,-15.521434,-15.521434,...,1.94,2.02,2.09,2.17,2.28,2.39,2.51,2.62,2.81,POINT (-71.12300 -18.09470)
3,GEDI02_A_2019108093620_O01965_01_T05338_02_003...,19650000100036040,coverage,BEAM0000,4.081639e+07,2019-04-18 09:53:13.311039649+00:00,0.759716,1,-15.521042,-15.521042,...,1.87,1.94,2.06,2.13,2.24,2.39,2.51,2.69,2.92,POINT (-71.12267 -18.09429)
4,GEDI02_A_2019108093620_O01965_01_T05338_02_003...,19650000100036041,coverage,BEAM0000,4.081639e+07,2019-04-18 09:53:13.319303647+00:00,0.727532,1,-15.520650,-15.520650,...,1.68,1.76,1.83,1.91,2.02,2.13,2.24,2.39,2.62,POINT (-71.12234 -18.09389)


## Upload to PostGIS database

In [9]:
from sqlalchemy import create_engine
import sqlalchemy as db
from src.constants import DB_CONFIG

engine = create_engine(DB_CONFIG, echo=False) 
gedi_l2a = db.Table("level_2a", db.MetaData(), autoload=True, autoload_with=engine)

<ipython-input-9-5dbaf001cc78>:6: SAWarning: Did not recognize type 'geometry' of column 'geometry'
  gedi_l2a = db.Table("level_2a", db.MetaData(), autoload=True, autoload_with=engine)


In [ ]:
for i, feather_file in enumerate(tqdm(feather_files[479:])):
    try:
        print(i+479)
        sample = gpd.read_feather(feather_file)
        sample[sample.quality_flag == 1]
        sample.to_postgis(name="level_2a", 
                          if_exists="append", 
                          con=engine, 
                          index=False, 
                          index_label="shot_number")
    except Exception as e:
        
        print(e)
        continue

## Load from PostGIS

### Runtime comparision after uploading only `feather_files[0]`

In [61]:
%%time
df = pd.read_sql(gedi_l2a.select(), con=engine)  # reads only data, not geometry

CPU times: user 4.92 s, sys: 318 ms, total: 5.24 s
Wall time: 10.6 s


In [62]:
%%time
df = gpd.read_postgis(gedi_l2a.select(), con=engine, geom_col="geometry")  # reads geometry as well

CPU times: user 6 s, sys: 360 ms, total: 6.36 s
Wall time: 11.2 s


In [66]:
%%time
sample = pd.read_feather(feather_files[0], columns=["granule_name"])  # read from feather format (no geometry)

CPU times: user 83.4 ms, sys: 7.15 ms, total: 90.5 ms
Wall time: 133 ms


In [68]:
%%time
sample = gpd.read_feather(feather_files[0], columns=["geometry"])  # read only geometry column from feather format

CPU times: user 714 ms, sys: 20.3 ms, total: 735 ms
Wall time: 737 ms


In [69]:
%%time
sample = gpd.read_feather(feather_files[0])

CPU times: user 732 ms, sys: 104 ms, total: 836 ms
Wall time: 773 ms


### Test out sql query

In [28]:
sql = "SELECT * FROM gedi_l2a"
df = gpd.read_postgis(sql, con=engine)

2021-06-10 13:25:13,029 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-06-10 13:25:13,030 INFO sqlalchemy.engine.Engine [cached since 169.5s ago] {'name': 'SELECT * FROM gedi_l2a'}
2021-06-10 13:25:13,036 INFO sqlalchemy.engine.Engine SELECT * FROM gedi_l2a
2021-06-10 13:25:13,037 INFO sqlalchemy.engine.Engine [raw sql] {}


ValueError: Query missing geometry column 'geom'